In [10]:
import os
import json

import openai
from openai import OpenAI

In [11]:
api_key = os.getenv("OPENAI_API_KEY_MA")
if api_key is None:
    raise ValueError(
        "API key not found. Please set the OPENAI_API_KEY environment variable."
    )

In [12]:
oai_client = OpenAI(api_key=api_key)
gpt_model = "gpt-4o-mini-2024-07-18"

In [13]:
def rephrase_text(text_to_rephrase, client, model):
    try:
        response = client.responses.create(
            model=model,
            input=[
                {
                    "role": "system",
                    "content": "You are an expert in paraphrasing text. Your task is to generate three distinct rephrasings of the provided text while maintaining the original meaning. Ensure that each rephrasing is unique and does not simply repeat the original text.",
                },
                {
                    "role": "user",
                    "content": f"Generate three paraphrases of the following text:\n\n{text_to_rephrase}\n\nDo not divert from the original meaning in any way. Use different words and sentence structures to convey the same message.\nMake sure to provide three distinct rephrasings.",
                },
            ],
            text={
                "format": {
                    "type": "json_schema",
                    "name": "rephrasings_list",
                    "strict": True,
                    "schema": {
                        "type": "object",
                        "properties": {
                            "rephrasings": {
                                "type": "array",
                                "description": "A list of rephrased versions of the original input.",
                                "items": {
                                    "type": "string",
                                    "description": "A single rephrased sentence.",
                                },
                            },
                        },
                        "required": ["rephrasings"],
                        "additionalProperties": False,
                    },
                }
            },
        )
        event = json.loads(response.output_text)
        return event
    except openai.BadRequestError as e:
        error_message = str(e)  # Extract the error message as a string
        print(f"Error: {error_message}")
        return {"error": error_message}

In [18]:
rephrase = rephrase_text(
    "The quick brown fox jumps over the lazy dog.",
    client=oai_client,
    model=gpt_model,
)

In [19]:
print(rephrase["rephrasings"])

['A fast brown fox leaps over a sleepy dog.', 'The swift brown fox bounds over the sluggish dog.', 'The agile brown fox springs over the drowsy dog.']
